1. Для 1 любого видео восстановить траекторию движения (t-вектор). Выполнить визуализацию. Определить параметры которые влияют на "точность" определения вектора t. 

Алгоритм:
1. Считываем по кадру из видео, каждый кадр преобразуем в черно-белый и уменьшаем в 9 раз.
2. Получаем особые точки и дескрипторы точек кадра.
3. Сопоставляем точки с двух соседних кадров по их дискрипторам.
3. Оставляем 20% точек для повышения точности.
4. Получаем оценочную матрицу по точкам двух кадров.
5. Сохраяем матрицы поворота и смемщения камеры.
6. Ключевые точки и дескрипторы текущего кадра сохраняем для дальнейших вычислений на следующей итерации.
7. Визуализирум траекторию положений камеры и ее направления в каждой точке.
8. Сохраняем массивы с матрицами углов и смещений в npz-файл, т к очередное получение данных матриц занимает много времени.

In [1]:
import plotly.graph_objects as gr
import numpy as np
import cv2

# function for create camera trajectory by video
def process_video(source_video: str):
    cap = cv2.VideoCapture(source_video)

    # resize frames to speed up simillar points detection 
    scale = 3
    scale_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / scale)
    scale_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / scale)

    # create SIFT detector and BFMatcher objects for all video frames
    sift_detector = cv2.SIFT_create()
    matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=True)

    # keypoints and descriptors from prev frame
    kps1, des1 = None, None

    # create_array with points of camera trajectory
    trajectory = np.array([[0, 0, 0]])

    # create general list with rotations matrix corresponds to each video frame
    rotations_list = [np.zeros((3, 3))]

    cam_matrix = np.eye(4)
    T = np.eye(4)

    while True:
        is_success, frame = cap.read()

        if is_success:
            # convert frame to gray and scale
            scaled_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 
                                      (scale_width, scale_height), cv2.INTER_CUBIC)
            # detect frame keypoints
            kps2, des2 = sift_detector.detectAndCompute(scaled_frame, None)

            if kps1 is not None:
                # match keypoints from 2 frames, get 20% from matched points for accuracy
                matches = matcher.match(np.array(des1, dtype=np.uint8), np.array(des2, dtype=np.uint8))
                matches = sorted(matches, key=lambda x: x.distance)[:int(0.2 * len(matches))]

                points1 = np.array([kps1[m.queryIdx].pt for m in matches])
                points2 = np.array([kps2[m.trainIdx].pt for m in matches])

                # calculate essential matrix to match camera positons between 2 frames
                e_mat, mask = cv2.findEssentialMat(points1, points2, method=cv2.LMEDS)
                _, R, t, _ = cv2.recoverPose(e_mat, points1, points2, mask=mask)

                T[:3, :3] = R
                T[:3, 3] = t.T

                cam_matrix = np.dot(cam_matrix, T)
                trajectory = np.vstack([trajectory, cam_matrix[:3, 3]])
                rotations_list.append(cam_matrix[:3, :3])

            kps1 = kps2
            des1 = des2
        else:
            break

    cap.release()

    return rotations_list, trajectory


In [2]:
# function for vizualize cemera trajectory and direction
def visualize_trajectory(trajectory, rotation):
    fig = gr.Figure()

    # add trajectory trace
    fig.add_trace(gr.Scatter3d(x=trajectory[:, 0], y=trajectory[:, 1], 
                               z=trajectory[:, 2],
                               marker=dict(size=3, color='purple')))

    # add camera orientation traces
    for (t, r) in zip(trajectory, rotation):
        point2 = t + r[:, 2]
        fig.add_trace(gr.Scatter3d(x=[t[0], point2[0]],
                                   y=[t[1], point2[1]],
                                   z=[t[2], point2[2]],
                                   mode='lines',
                                   line=dict(width=2, color='red')))

    fig.update_layout(title='Camera motion', showlegend=False)
    fig.show()


In [ ]:
source_video = '/media/vik/SamsungSSD7/Courses/peleng-cources/HW_10/videos/concat_video.mp4'

# get camera rotations and trajectory
rotations_list, trajectory = process_video(source_video)

# visualize camera motion
visualize_trajectory(trajectory, rotations_list)


In [5]:
npz_filename = '/media/vika/SamsungSSD7/Courses/peleng-cources/HW_10/videos/video_data.npz'


In [14]:
# save camera rotations and trajectory to npz-file
np.savez(npz_filename, R=rotations_list, trajectory=trajectory)


In [6]:
# load arrays from npz-file
data = np.load(npz_filename)
trajectory = data['trajectory']
rotations_list = data['R']


2. Использовать решение на базе нейронных сетей. Любые идеи. 
3. ***slam прикрутить. 